# Modelos basados en estados

## Problemas de búsqueda

En este bloque discutiremos en general modelos de inteligencia artificial basados en estados y en particular problemas de búsqueda.

Encontrar una ruta de un lugar a otro es uno de los ejemplos clásicos de problemas de búsqueda.  Tenemos de entrada un mapa, un punto de origen y un punto destino.  El objetivo es producir como salida una secuencia de acciones.

Consideremos el siguiente mapa

<center>
    <img src="./img00.png" />
</center>

Considerando que el punto de inicio es el edificio 3K4 en la UNISON y el punto final es La Biblioteca, una solución al problema de encontrar una ruta entre los dos puntos sería la siguiente:

- Dirígete al este por Av Luis Donaldo Colosio Murrieta por 190 metros
- Gira a la derecha hacia Calle Av. Rosales/Guaymas-Hermosillo/Hermosillo - Guaymas por 72 metros
- Gira a la derecha con dirección a Calle Av. Rosales/Guaymas - Hermosillo/Hermosillo - Guaymas por 74 metros
- Gira a la izquierda con dirección a Av. Dr. Alberto G. Noriega por 30 metros
- El destino está a la derecha

Al seguir estas instrucciones, llegamos al destino.

<center>
    <img src="./img01.png" />
</center>

¿Cómo podemos evaluar qué tan buenas son las secuencias de acciones?

- Distancia recorrida
- Tiempo de llegada
- Caminos más seguros
- Recorrido más pictórico

Otro contexto es en planeación de movimiento de un robot.  Esto puede ir desde el desplazamiento del robot en el espacio, hasta la activación coordinada de cada motor en el brazo robótico para abrir una puerta.

Uno de los algoritmos de búsqueda más importantes que discutiremos, el $A^\ast$, fue desarrollado para la operación de los primeros robots inteligentes.


<center>
    <img src="./img02.gif" />
</center>

En este ejemplo de planeación en robots, ¿Cómo podemos evaluar un plan?

- Mas rápido de ejecutar
- Mas eficiente en gasto de energía
- Mas seguro de realizar
- Mas expresivo

Podemos complicar más el ejemplo, en lugar de construir planes para un robot, podríamos construir planes para toda una flotilla de robots para eficientar la organización de los almacenes en Correos de México.

<center>
    <img src="./img03.jpg" />
</center>

Problemas de búsqueda también aparecen en varios juegos como el cubo de Rubik o el Sudoku.

<center>
    <img src="./img04.png" />
</center>

¿Qué forma tendrían las acciones?

¿Qué forma tendrían los objetivos?

### Más allá de reflejos

En un problema de búsqueda, de cierta forma, seguimos construyendo un predictor $f$ que toma una entrada $x$. Pero $f$ ahora regresa una *secuencia de acciones* no solo una acción.

En los problemas de clasificación:
<center>
    <img src="./img05.png" />
</center>

En los problemas de búsqueda:
<center>
    <img src="./img06.png" />
</center>

### ¿Qué discutiremos en este bloque?

- Cómo modelar problemas de búsqueda
- Búsqueda en árboles
- Programación dinámica
- Búsqueda de costo uniforme
- Algoritmo A*
- Cómo lidiar con problemas parcialmente especificados

## Modelación de problemas de búsqueda

¿Recuerdan el acertijo del lobo, la cabra y la col?

*Un día, un granjero fue al mercado y compró un lobo, una
cabra y una col. Para volver a su casa tenía que cruzar
un río. El granjero dispone de una barca para cruzar a
la otra orilla, pero en la barca solo caben él y una de sus
compras.*


*Si el lobo se queda solo con la cabra, se la come.  Si la cabra se queda sola con la col, se la come.*

*El reto del granjero es cruzar él mismo y dejar sus compras a la otra orilla del río, dejando cada compra intacta.*

---

<center>
    <img src="./img07.png" />
</center>

Definimos nuestro problema de búsqueda especificando:
- Estado inicial: $s_0$
- Posibles acciones en el estado $s$: $\mathrm{Actions}(s)$
- Costo de realizar una acción $a$ en un estado $s$: $\mathrm{Cost}(s, a)$
- Estado al que llegamos después de aplicar una acción $a$ en un estado $s$: $\mathrm{Succ}(s, a)$
- Predicado para determinar si un estado $s$ es final: $\mathrm{IsEnd}(s)$

Codificamos un estado representando con `F` al granjero, `C` a la col, `G` a la cabra, `W` al lobo y `||` al río.

El estado inicial es `FCGW||`,

El universo de acciones consiste de ocho, ya que en la barca caben a lo más dos entidades y una de ellas debe ser el granjero, estas representan qué entidades se mueven al otro lado del rio en la barca y en qué dirección:
- `F>`
- `F<`
- `FC>`
- `FC<`
- `FG>`
- `FG<`
- `FW>`
- `FW<`

Se omiten las acciones que nos regresan a un estado previo.
Consideramos adicionalmente que cada acción tiene un costo unitario.

La propuesta es construir un árbol de búsqueda donde la raíz sea el estado inicial y las hojas los estados finales. Cada arista saliente de un nodo $s$ corresponde a una posible acción en $\mathrm{Actions}(s)$ que pueda realizarse en el estado $s$. Las aristas son etiquetadas por una acción yb un costo, denotado $a:\mathrm{Cost}(s,a)$.

<center>
    <img src="./img08.png" />
</center>

Una diferencia fundamental del concepto de árbol de búsqueda en IA y Estructuras de Datos es que en IA no construímos el árbol en memoria, usualmente se construye en memoria una trayectoria del estado inicial a un estado final con algunas rutas del árbol posiblemente sin explorar.

**Para la tarea:** Modela el juego de las torres de Hanói como un problema de búsqueda.

In [1]:
def init(num_discos):
    torre1 = list(range(num_discos, 0, -1))
    
    torre2 = []
    torre3 = []
    
    return torre1, torre2, torre3

def Actions(state):
    torre1, torre2, torre3 = state
    
    acciones = []
    if torre1:
        if not torre2 or torre1[-1] < torre2[-1]:
            acciones.append(('torre 1', 'torre 2'))
        if not torre3 or torre1[-1] < torre3[-1]:
            acciones.append(('torre 1', 'torre 3'))
    if torre2:
        if not torre1 or torre2[-1] < torre1[-1]:
            acciones.append(('torre 2', 'torre 1'))
        if not torre3 or torre2[-1] < torre3[-1]:
            acciones.append(('torre 2', 'torre 3'))
    if torre3:
        if not torre1 or torre3[-1] < torre1[-1]:
            acciones.append(('torre 3', 'torre 1'))
        if not torre2 or torre3[-1] < torre2[-1]:
            acciones.append(('torre 3', 'torre 2'))
    
    return acciones

def Cost(state, actions):
    return 1

def Succesor(state, action):
    torre1, torre2, torre3 = state
    torre_origen, torre_destino = action
    
    torre_origen_copy = torre1[:] if torre_origen == 'torre 1' else torre2[:] if torre_origen == 'torre 2' else torre3[:]
    torre_destino_copy = torre1[:] if torre_destino == 'torre 1' else torre2[:] if torre_destino == 'torre 2' else torre3[:]
    
    if not torre_origen_copy:
        return state
    
    disco = torre_origen_copy.pop()
    torre_destino_copy.append(disco)
    
    if torre_origen == 'torre 1':
        torre1 = torre_origen_copy
    elif torre_origen == 'torre 2':
        torre2 = torre_origen_copy
    else:
        torre3 = torre_origen_copy
        
    if torre_destino == 'torre 1':
        torre1 = torre_destino_copy
    elif torre_destino == 'torre 2':
        torre2 = torre_destino_copy
    else:
        torre3 = torre_destino_copy
        
    return (list(torre1), list(torre2), list(torre3))

def isEnd(state):
    return True if state == ([], [], [3, 2, 1]) else False

In [2]:
state = init(3)

In [3]:
state

([3, 2, 1], [], [])

In [7]:
state = Succesor(state, Actions(state)[1])
state = Succesor(state, Actions(state)[0])
state = Succesor(state, Actions(state)[2])
state = Succesor(state, Actions(state)[0])
state = Succesor(state, Actions(state)[0])
state = Succesor(state, Actions(state)[2])
state = Succesor(state, Actions(state)[1])

In [8]:
state

([], [], [3, 2, 1])

In [9]:
isEnd(state)

True

**Para la tarea:** Escribe un programa de Python que genere un tablero de Sudoku resuelto de forma aleatoria de tamaño $9\times9$.  No busques algoritmos para lograr esto, en su lugar, plantea una idea clave para resolver el problema y asegúrate que tu implementación sea lo más clara y entendible que puedas.

In [61]:
import random, numpy as np

import copy

def shuffle_board(board):
    shuffled_board = copy.deepcopy(board)
    solver(shuffled_board)
    while True:
        for i in range(len(board)):
            for j in range(len(board[0])):
                if random.random() > 0.5:
                    shuffled_board[i][j] = 0
        solver(shuffled_board)
        if has_unique_solution(shuffled_board):
            return shuffled_board
        
def has_unique_solution(board):
    solved_board = copy.deepcopy(board)
    if solver(solved_board):
        for i in range(len(board)):
            for j in range(len(board[0])):
                if board[i][j] == 0:
                    temp_board = copy.deepcopy(board)
                    temp_board[i][j] = 1
                    if solver(temp_board):
                        return False
        return True
    else:
        return False


def initial_board(n, m):
    board = [[0 for j in range(m)] for i in range(n)]
    return board

def is_valid_move(board, row, col, n):
    
    # Comprobar fila y columna
    if any(board[col][i] == n for i in range(len(board))):
        return False
    if any(board[i][row] == n for i in range(len(board[0]))):
        return False

    # Comprobar subcuadrícula
    subrow_start = (row // int(len(board[0])**0.5)) * int(len(board[0])**0.5)
    subcol_start = (col // int(len(board)**0.5)) * int(len(board)**0.5)
    
    if any(board[subcol_start + i][subrow_start + j] == n
           for i in range(int(len(board)**0.5)) for j in range(int(len(board[0])**0.5))):
        return False
    
    return True

def solved(board):
    for col in range(len(board)):
        for row in range(len(board[0])):
            if board[col][row] == 0:
                for n in random.sample(range(1, len(board) + 1), len(board)):
                    if is_valid_move(board, row, col, n):
                        board[col][row] = n
                        if solved(board):
                            return True
                        board[col][row] = 0
                return False
    return True

while True:
    board = initial_board(9, 9)
    shuffle_board(board)
    if solved(board):
        print(np.matrix(board))
        break

[[7 1 5 6 9 8 4 2 3]
 [8 9 2 4 7 3 5 6 1]
 [4 6 3 5 2 1 8 9 7]
 [6 4 7 9 5 2 1 3 8]
 [9 3 8 7 1 6 2 4 5]
 [5 2 1 3 8 4 9 7 6]
 [1 5 9 2 3 7 6 8 4]
 [3 8 6 1 4 9 7 5 2]
 [2 7 4 8 6 5 3 1 9]]


In [55]:
solved = solved(board)
solved

True

**Demostración en clase:**

Consideremos una calle con cuadras numeradas del $1$ a $n$.

Caminar de la cuadra $s$ a la $s+1$ nos toma $1$ minuto.

Tomar un camión mágico de $s$ a $2s$ nos toma $2$ minutos.

Comenzamos en la cuadra $1$, queremos llegar a la cuadra $n$.

¿Cómo podemos hacer esta travesía en el menor tiempo posible?

In [11]:
class TransportationProblem(object):
    def __init__(self, n):
        self.n = n
    
    def initialState(self):
        return 1
    
    def isEnd(self, state):
        return state == self.n
    
    def actions(self, state):
        moves = []
        if state + 1 <= self.n:
            moves.append('walk')
        if state * 2 <= self.n:
            moves.append('tram')
        return moves
    
    def cost(self, state, action):
        if action == 'walk':
            return 1
        if action == 'tram':
            return 2
    
    def successor(self, state, action):
        if action == 'walk':
            return state + 1
        if action == 'tram':
            return state * 2

In [12]:
def edges(problem, state):
    return [
        (action,
         problem.successor(state, action),
         problem.cost(state, action))
        for action in problem.actions(state)
    ]

In [13]:
problem = TransportationProblem(n = 4)

In [14]:
print(edges(problem, 1))

[('walk', 2, 1), ('tram', 2, 2)]


In [15]:
print(edges(problem, 2))

[('walk', 3, 1), ('tram', 4, 2)]


In [16]:
print(edges(problem, 3))

[('walk', 4, 1)]


## Búsqueda en árboles

Supongamos que se nos presenta un problema de búsqueda.

¿Cómo construimos un algoritmo para encontrar una trayectoria de mínimo costo entre el estado inicial y alguno de los estados finales?

Consideremos todas las posibles trayectorias. A esto le llamamos **búsqueda con backtracking**.

<center>
    <img src="./img09.gif" />
</center>

```python
def backtrackingSearch(state, path):
    if IsEnd(state):
        updateMinCost(path)
    for action in Actions(state):
        next_state = Successor(state, action)
        next_cost = Cost(state, action)
        next_path = updatePath(path, next_state, next_cost)
        backtrackingSearch(next_state, next_path)
    return minCostPath
```

La búsqueda con backtracking realiza un recorrido del árbol de búsqueda, considerando la trayectoria desde el estado inicial al estado actual.  Una vez que se alcanza un estado final, se considera la trayectoria resultante como candidata a ser la trayectoria de menor costo.

Supongamos que el árbol de búsqueda tiene una profundidad máxima $D$ y que hay $b$ acciones en cada estado.

*¿Cuál es la complejidad de memoria del algoritmo?*

$$O(D)$$

Ya que la búsqueda solo requiere almacenar la trayectoria candidata, la cuál en el peor de los casos tiene longitud $D$.

*¿Cuál es la complejidad de tiempo del algoritmo?*

Por cada nodo del árbol se realiza una cantidad constante de operaciones, por lo que el tiempo de ejecución es proporcional a la cantidad de nodos del árbol.

$$1+b+b^2+b^3+\dots+b^D = \frac{b^{D+1}-1}{b-1} = O(b^D)$$

**Ejercicio en clase:** Considera una modificación de la búsqueda con backtracking que incorpora una estrategia voraz (*greedy*), en lugar de considerar todas las posibles acciones, solo elige aquella que produce el costo mínimo en el estado actual. ¿Encontraríamos la trayectoria de costo mínimo? Si tu respuesta es afirmativa, demuestra por qué. Si tu respuesta es negativa, presenta un contraejemplo.

**Ejercicio en clase:** Considera la siguiente implementación del algoritmo de búsqueda con backtracking para el problema del camión mágico.

Encuentra la mejor solución al problema

In [17]:
def backtrackingSearch(problem):
    bestCost = float('+inf')
    bestPath = None
    def findBest(state, path, cost):
        nonlocal bestCost
        nonlocal bestPath
        if problem.isEnd(state):
            if cost < bestCost:
                bestCost = cost
                bestPath = path
            return
        for action, next_state, next_cost in edges(problem, state):
            findBest(next_state,
                     path + [(action, next_state, next_cost)],
                     cost + next_cost)
    findBest(state = problem.initialState(),
             path = [],
             cost = 0)
    return bestCost, bestPath

In [18]:
backtrackingSearch(problem)

(3, [('walk', 2, 1), ('walk', 3, 1), ('walk', 4, 1)])

**Para la tarea:** Escribe una implementación de `backtrackingSearch` que sea funcionalmente equivalente a la implementación de arriba sin utilizar variables fuera del ámbito local de una función.

In [19]:
def backtrackingSearchNonLocal(problem):
    def findBest(state, path, cost, bestCost, bestPath):
        if problem.isEnd(state):
            if cost < bestCost:
                bestCost = cost
                bestPath = path
            return bestCost, bestPath
        for action, next_state, next_cost in edges(problem, state):
            new_path = path + [(action, next_state, next_cost)]
            new_cost = cost + next_cost
            bestCost, bestPath = findBest(next_state, new_path, new_cost, bestCost, bestPath)
        return bestCost, bestPath
    initial_state = problem.initialState()
    bestCost, bestPath = findBest(initial_state, [], 0, float('+inf'), None)
    return bestCost, bestPath


In [20]:
backtrackingSearchNonLocal(problem)

(3, [('walk', 2, 1), ('walk', 3, 1), ('walk', 4, 1)])

---
La búsqueda con backtracking encuentra la trayectoria de mínimo costo, siempre y cuando el árbol de búsqueda sea finito, sin embargo, hay situaciones donde podemos ser más rápidos. Para lograr esto, necesitamos incorporar algunas suposiciones adicionales.

Consideremos que todas las acciones tienen un costo de cero en todos los estados.  En otras palabras, lo que nos importa es encontrar una secuencia válida de acciones que nos lleven del estado inicial a algún estado final.  Cualquera de estas secuencias tiene un costo mínimo de cero.

<center>
    <img src="./img10.svg" />
</center>

En este caso, podemos simplemente regresar la primer trayectoria que alcance un estado final, sin preocuparnos por los costos. El algoritmo resultante es la **búsqueda a lo profundo** llamada en inglés depth-first search (DFS).

**Ejercicio en clase:** ¿Cuál es la complejidad de memoria de DFS? ¿Cuál es la complejidad en tiempo de DFS? Si no nos importan los costos, ¿Es mejor DFS que búsqueda con backtracking?

**Ejercicio en clase:** Implementa DFS para resolver el problema del camión mágico.

---

La suposición que incorporamos para obtener búsqueda a lo profundo es muy fuerte, que los costos no importen.  Ahora consideramos la **búsqueda a lo ancho** llamda en inglés breadth-first search (BFS) que parte de la idea de encontrar una trayectoria válida pero buscando trayectorias de menor a mayor tamaño.  Esto es como si consideráramos $\mathrm{Cost}(s, a) = 1$ para todo estado $s$ y acción $a$.

<center>
    <img src="./img11.svg" />
</center>

La búsqueda a lo ancho utiliza una cola de estados pendientes de explorar.  Saca un estado de la cola y luego encola sus sucesores.  Esta busqueda considera todas las trayectorias que consisten de una arista, luego todas las de dos, luego todas las de tres, etc., hasta que encuentra una trayectoria que llega a un estado final.

**Ejercicio en clase:** Si una solución más corta tiene $d$ acciones, ¿Cuál es la complejidad en tiempo de BFS? ¿Cuál es la complejidad en espacio de BFS? ¿Es mejor BFS que DFS para árboles de búsqueda finitos?

**Ejercicio en clase:** Implementa BFS para resolver el problema del camión mágico.

---

¿Podemos mejorar las propuestas anteriores?

Podemos modificar el DFS para que se detenga al llegar a cierta profundidad. De tal manera que al invocat el DFS modificado con profundidad máxima $d$ nos dice si existe una trayectoria de longitud a lo más $d$, lo cuál resulta en $O(d)$ en espacio y $O(b^d)$ en tiempo.

Luego podemos invocar este método con profundidades $1$, $2$, $3$, $\dots$ hasta que encontremos una solución o nos demos por vencidos.

Este algoritmo es llamado **búsqueda en profundidad iterativa**, en inglés depth-first search with iterative deepening (DFS-ID).

Si una solución más corta tiene $d$ acciones:

*¿Cuál es la complejidad en tiempo de DFS-ID?*

Primero analizamos cuál es la complejidad en tiempo para una búsqueda a lo profundo con profundidad máxima $d$. Ya que en el peor de los casos se exploran todos los nodos de un árbol con profundidad $d$ y factor de ramificación $b$, esta complejidad es $O(b^d)$.

$$O(b^0) + O(b^1) + O(b^2) + \dots + O(b^d) = O(b^d)$$

*¿Cuál es la complejidad en espacio de DFS-ID?*

Solo almacenamos la trayectoria candidata actual, la cuál consiste de a lo más $d$ acciones.

$$O(d)$$

**Ejercicio en clase:** Implementa DFS-ID para resolver el problema del camión mágico.

---

En resumen, no podemos evitar la complejidad en tiempo exponencial, pero podemos obtener complejidad en espacio lineal. El espacio a veces es mas importante que el tiempo en problemas de búsqueda ya que la memoria no puede "crecer" mágicamente, sin embargo, si nos podemos dar el lujo de correr el programa por más tiempo, o incluso paralelizarlo para que cada procesador realiza la búsqueda en un subárbol diferente.

| **Algoritmo** | **Costo de acción** | **Espacio** | **Tiempo** |
|---------------|---------------------|-------------|------------|
| Backtracking  | cualquiera          | $O(D)$      | $O(b^D)$   |
| DFS           | cero                | $O(D)$      | $O(b^D)$   |
| BFS           | uno                 | $O(b^d)$    | $O(b^d)$   |
| DFS-ID        | uno                 | $O(d)$      | $O(b^d)$   |

##  Programación dinámica

Pensemos en el árbol de búsqueda para el problema del camión mágico suponiendo que $n$ es muy grande.

Iniciamos en la cuadra $1$, si caminamos nos vamos a la cuadra $1+1=2$ y si tomamos el camión mágico nos vamos a la cuadra $1\times2=2$. En ambos casos quedamos en la cuadra $2$.

<center>
    <img src="./img12.svg" />
</center>

A partir de este estado, la secuencia de acciones con el menor costo para llegar a la cuadra $n$ es la misma. El costo de la secuencia total será distinto, ya que caminar es menos costoso que tomar el camión. Pero eso no es lo importante ahorita.

Enfaticemos los siguiente: Si la primera acción es `walk`, la mejor secuencia para el resto de las acciones será igual a la que obtendríamos si la primera acción es `tram`.

<center>
    <img src="./img13.svg" />
</center>

En los algoritmos de búsqueda que hemos discutido, los subárboles correspondientes a tomar la acción `walk` y `tram` se exploran de manera independiente. Sin embargo, realizan escencialmente el mismo cálculo.

Si podemos eliminar esta redundancia de cálculos, podremos reducir los cálculos mas o menos a la mitad... a menos que...

<center>
    <img src="./img14.gif" style="width: 100%" />
</center>

¡Podemos reducir la complejidad de tiempo! La redundancia aparece en otras partes del árbol. 

<center>
    <img src="./img15.svg" />
</center>

Vamos a conceptualizar esta observación de la siguiente manera. El objetivo del problema es encontrar la secuencia de acciones de menor costo que nos llevan de la cuadra $1$ a la cuadra $n$, es decir, del estado inicial a un estado final.

Sea $s$ un estado cualquiera, el menor costo al que podemos aspirar estando en $s$ lo denotamos $\mathrm{FutureCost}$, definido como
$$\mathrm{FutureCost}(s) = \begin{cases}
0 & \text{si } \mathrm{IsEnd}(s) \\
\min_{a\in\mathrm{Actions}(s)}\{ \mathrm{Cost}(s, a) + \mathrm{FutureCost}(\mathrm{Succ}(s, a))\} & \text{en otro caso}
\end{cases}$$

**Discutir en clase:** Considera el problema del camión mágico para $n=8$, ¿Cuántos nodos tiene el árbol de búsqueda?

Consideremos el problema del camión mágico para $n=8$ y supongamos que se nos provee un vector $C$ de $8$ elementos donde $C_i = \mathrm{FutureCost}(i)$.

Entonces podemos resolver el problema de búsqueda de la siguiente manera:

In [21]:
def dynamicProgramming(problem, C):
    state = problem.initialState()
    bestCost = C[state]
    bestPath = []
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
    return bestCost, bestPath

In [22]:
problem = TransportationProblem(n = 8)

dynamicProgramming(problem,
                   {1: 5, 2: 4, 3: 3, 4: 2, 5: 3, 6: 2, 7: 1, 8: 0})

(5, [('walk', 2, 1), ('walk', 3, 1), ('walk', 4, 1), ('tram', 8, 2)])

In [23]:
def futureCosts(problem):
    C = {}
    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for action, next_state, next_cost
                           in edges(problem, state))
        return C[state]
    futureCost(problem.initialState())
    return C

In [24]:
futureCosts(problem)

{8: 0, 7: 1, 6: 2, 5: 3, 4: 2, 3: 3, 2: 4, 1: 5}

¿Cómo se compara la búsqueda con backtracking y la programación dinámica?

In [25]:
problem = TransportationProblem(n = 100)

In [26]:
%%timeit -r 1 -n 1
print(backtrackingSearch(problem))
print()

(13, [('walk', 2, 1), ('walk', 3, 1), ('tram', 6, 2), ('tram', 12, 2), ('tram', 24, 2), ('walk', 25, 1), ('tram', 50, 2), ('tram', 100, 2)])

326 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
%%timeit -r 1 -n 1
print(dynamicProgramming(problem, futureCosts(problem)))
print()

(13, [('walk', 2, 1), ('walk', 3, 1), ('tram', 6, 2), ('tram', 12, 2), ('tram', 24, 2), ('walk', 25, 1), ('tram', 50, 2), ('tram', 100, 2)])

797 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Para la tarea:** Observa que tanto la implementación de `futureCosts` como de `dynamicProgramming` iteran sobre las aristas del estado actual. Programa una mejor implementación combinando las ideas de ambas funciones.

In [35]:
def dynamicProgrammingWithFutureCosts(problem):
    C = {}
    bestPath = []
    
    def futureCost(state):
        if state in C:
            return C[state]
        if problem.isEnd(state):
            C[state] = 0
        else:
            C[state] = min(next_cost + futureCost(next_state)
                           for action, next_state, next_cost
                           in edges(problem, state))
        print(C.values())
        return C[state]
    
    state = problem.initialState()
    bestCost = futureCost(state)
    while not problem.isEnd(state):
        for action, next_state, next_cost in edges(problem, state):
            if C[state] == next_cost + C[next_state]:
                bestPath.append((action, next_state, next_cost))
                state = next_state
                break
        else:
            raise Exception("No outgoing edges with minimum cost found!")
    return bestCost, bestPath

In [37]:
%%timeit -r 1 -n 1
dynamicProgrammingWithFutureCosts(problem)
print()

dict_values([0])
dict_values([0, 1])
dict_values([0, 1, 2])
dict_values([0, 1, 2, 3])
dict_values([0, 1, 2, 3, 4])
dict_values([0, 1, 2, 3, 4, 5])
dict_values([0, 1, 2, 3, 4, 5, 6])
dict_values([0, 1, 2, 3, 4, 5, 6, 7])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dict_values([0, 1, 2, 3, 4, 5, 6, 7

---

El algoritmo de programación dinámica que discutimos realiza una búsqueda con backtracking pero utiliza la técnica llamada *memoización* (no confundir con memorización, *memoización* viene de **memorandum** no de **memorizar**, aunque ambas palabras vienen de la misma raíz latina).

Una limitación de esta técnica es que la gráfica de estados debe ser acíclica. Si tiene ciclos, entonces el cálculo de un costo futuro para $s$ depende de un $s'$ que a su vez puede depender de $s$ que es precisamente el que queremos calcular.

En el ejemplo que discutimos el costo depende de la cuadra actual. Sin embargo, podemos modelar problemas en donde el estado incluya más información.

Un **estado** es un resumen de todas las acciones pasadas que sean suficientes para elegir acciones futuras de manera óptima. El truco es decidir qué información del pasado podemos olvidar. Entre más olvidemos, menos estados y más eficiente nuestro programa.

Veamos un problema:


---

*Encuentra el camino de costo mínimo de la ciudad $1$ a la ciudad $n$, únicamente moviéndose hacia adelante. Cuesta $c_{ij}$ ir de $i$ a $j$.*

*Adicionalmente, no podemos visitar tres ciudades impares al hilo.*

Si solo nos interesara la ciudad actual, tendríamos un árbol de búsqueda como el siguiente y podríamos aplicar los algoritmos que ya discutimos igualito:

<center>
    <img src="./img18.png" />
</center>

En particular, utilizando programación dinámica, podemos resolver el problema construyendo la gráfica de dependencias para el cálculo del costo futuro y hacer que nuestro programa *vuele*.

<center>
    <img src="./img19.png" />
</center>

Sin embargo, para satisfacer la restricción adicional esto no es suficiente. Cuando nos intentemos mover a la siguiente ciudad, no podremos saber si honramos la restricción porque no sabemos de qué ciudad venimos. Entonces podemos incorporar la ciudad previa a nuestro estado, cada estado sería entonces una pareja de ciudades.

Esto haría que nuestro algoritmo funcionara, sin embargo la cantidad de estados en nuestro problema ahora es $n^2$.

Podemos mejorar esto recordando únicamente la paridad de la ciudad previamente visitada, por lo que los estados serían parejas cuyo primer elemento es una etiqueta *odd* o *even* y cuyo segundo elemento es la ciudad actual.

¡¡Esto nos reduce la cantidad de estados de $n^2$ a $2n$!!

<center>
    <img src="./img20.png" />
</center>

Consideremos que estamos en el estado $(\mathrm{odd},3)$.

A pesar de que venimos de una ciudad impar, podemos movernos al estado $(\mathrm{odd},4)$ ya que $4$ es par y por lo tanto la paridad de la secuencia de estados sería *impar*, *impar*, *par*.
Sin embargo, no podemos movernos al estado $(\mathrm{odd},7)$ ya que $7$ es impar y por lo tanto la paridad de la secuencia de estados sería *impar*, *impar*, *impar*.

---

Hasta este punto nos hemos expandido nuestro repertorio de algoritmos para resolver una amplia variedad de problemas de manera eficiente.

Pero sé lo que están pensando... La programación dinámica funciona solo cuando la gráfica de estados es acíclica... ¿Qué pasa si tengo un problema donde la gráfica tiene ciclos?

## Búsqueda de costo uniforme

Quizá algunos de ustedes ya tienen preparada una respuesta para la pregunta anterior.

> "Es prácticamente imposible enseñar buena programación a estudiantes que han tenido una exposición previa a BASIC: como programadores potenciales, están mentalmente mutilados más allá de toda esperanza de regeneración"

> "El uso de COBOL paraliza la mente; por lo tanto, su enseñanza debe considerarse un delito penal"

> "Las pruebas de programas se pueden usar para mostrar la presencia de errores, pero nunca para mostrar su ausencia"

> "La pregunta de si una computadora puede pensar no es más interesante que la pregunta de si un submarino puede nadar"

> "El progreso solo es posible si nos entrenamos para pensar en programas sin pensar en ellos como piezas de código ejecutable"

> "Si en física hay algo que no entiendes, siempre puedes esconderte detrás de las profundidades desconocidas de la naturaleza. Siempre puedes culpar a Dios. Tú no lo hiciste tan complejo. Pero si tu programa no funciona, hay nadie detrás de quien esconderse. No puedes esconderte detrás de una naturaleza obstinada. Si no funciona, la regaste"

> "No me culpes por el hecho de que la programación competente, ya que la veo como una posibilidad intelectual, será demasiado difícil para el 'programador promedio'. No debes caer en la trampa de rechazar una técnica quirúrgica porque está más allá del capacidades del barbero en su tienda a la vuelta de la esquina"

> "La programación orientada a objetos es una idea excepcionalmente mala que solo podría haberse originado en California"

> "El esfuerzo de usar máquinas para imitar la mente humana siempre me ha parecido bastante tonto. Prefiero usarlas para imitar algo mejor"

<center>
    <img src="./img21.jpg" />
</center>

El algoritmo de Dijkstra para encontrar rutas más cortas. Cuando nos refiramos a la *búsqueda de costo uniforme* (UCS) puedes pensar en Dijkstra.

Pero primero, veamos un video de cómo se comporta este algoritmo:

In [62]:
from IPython.display import Video
Video("./vid00.mp4", html_attributes='controls style="width: 100%"')

Recordemos que utilizamos programación dinámica para caluclar el costo futuro de cada estado $s$, es decir, el costo del camino de costo mínimo de $s$ a un estado final.

Podemos definir otro costo análogo $\mathrm{PastCost}(s)$, el costo del camino de mínimo costo del estado inicial hasta $s$. Si en lugar de conocer el sucesor $\mathrm{Succ}(s,a)$ tuviéramos los predecesores (pensemos en invertir la dirección de las aristas en la gráfica), entonces podríamos definir un programa dinámico para calcular el $\mathrm{PastCost}(s)$ de todo estado $s$.

<center>
    <img src="./img22.png" />
</center>

La idea central de la UCS es ordenar de manera creciente los estados de acuerdo a su costo pasado. Para poder hacer esto de manera eficiente, debemos incorporar la suposición de que todos los costos de las acciones son no-negativos (pueden ser cero). Si tuviéramos costos negativos, podríamos inscribirnos al curso de análisis de redes con Irene y aprender sobre el algoritmo de [Bellman-Ford](https://es.wikipedia.org/wiki/Algoritmo_de_Bellman-Ford).

La idea detrás de la UCS es partir el conjunto de estados en tres conjuntos disjuntos:
- Los estados explorados (comienza vacío)
- Los estados en la frontera (comienza con el estado inicial)
- Los estados no explorados (comienza con todos los estados excepto el inicial)

A lo largo del algoritmo, vamos a mover estados del conjunto de no explorados a la frontera y de la frontera a los explorados.

Conforme ampliamos nuestra exploración, se satisfacen las siguientes invariantes:
- Los tres conjuntos son disjuntos y su unión es todos los estados
- Conocemos los caminos de costo mínimo de todos los estados en el conjunto de explorados.

Veamos un ejemplo concreto de cómo funciona el algoritmo.

<center>
    <img src="./img23.png" style="width: 100%" />
</center>

**Para la tarea:** ¿Por qué el algoritmo de Dijkstra no puede trabajar con pesos negativos? ¿Qué pasa si le sumamos a todos los pesos el peso mínimo de la gráfica? Presenta una gráfica dirigida ponderada en donde esta "solución" no funciona.

**Solucion**

La razón por la cual no puede trabajar con pesos negativos es porque el algoritmo se basa en la idea de que, en cada iteración, el nodo más cercano al origen es elegido y su distancia es actualizada. Si hay pesos negativos en el grafo, el algoritmo podría encontrar un camino más corto a través de un ciclo negativo y entrar en un bucle infinito, ya que siempre habría un camino más corto al seguir el ciclo.

Si le sumamos a todos los pesos el peso mínimo de la gráfica, entonces ya no habría pesos negativos en la gráfica y el algoritmo de Dijkstra podría funcionar correctamente. Sin embargo, esta solución no siempre es óptima y puede afectar la precisión de los resultados, especialmente en gráficos grandes o complejos. Además, esta técnica solo funciona si el peso mínimo de la gráfica es no negativo. Si la gráfica contiene pesos negativos, entonces esta técnica no resolvería el problema y se necesitaría una solución diferente.

## Algoritmo $A^*$